# Install

In [2]:
# ! pip install -U agno
# ! pip install -U  python-dotenv rich
# ! pip install -U openai groq
# ! pip install sqlalchemy
# stop

# Load Secrets

In [4]:
# Import necessary libraries
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the API keys from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
gro_api_key = os.getenv('Groq_API_KEY')

# Print the keys (optional, for testing purposes)
print(f"OpenAI API Key: {bool(openai_api_key)}")
print(f"Gro API Key: {bool(gro_api_key)}")


OpenAI API Key: True
Gro API Key: True


# Load QnA from Google Sheet

In [6]:
import time
import pandas as pd

csv_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRJiwSeaHg2FAU9Z9Sgis0U5uQFbjoQNiyooQatqlabHQRZc0NhfJHuKec59j2l0xr7wd56gEuGy6ra/pub?output=csv"
qa_df = pd.read_csv(csv_url)
# Convert the DataFrame into the desired qa_data format
qa_data = [
    {
        "question": row["question"],
        "answer": row["answer"]
    }
    for _, row in qa_df.iterrows()
]

# Display qa_data to verify the output
# print(qa_data)

# OpenAI Agents

In [8]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.models.groq import Groq
from pydantic import BaseModel, Field
import pandas as pd

In [9]:
from pydantic import BaseModel, Field
from textwrap import dedent
from typing import Dict

class PersonalityTraits(BaseModel):
    positive: Dict[str, int] = Field(..., description="Positive personality traits with values from 0 to 100")
    negative: Dict[str, int] = Field(..., description="Negative personality traits with values from 0 to 100")
    quote: str = Field(..., description="A motivational or descriptive quote based on the analysis")


In [10]:
# Record start time
start_time = time.time()
# Create an OpenAI agent with PersonalityTraits response model
openai_agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    response_model=PersonalityTraits,
    instructions=dedent("""\
        You are a personality test expert, a psychologist, psychometrician, or organizational psychology specialist who develops, administers, and interprets personality tests.
        Based on given question/answer you find out persona traits and score them. You also generate motivational quote based on the persona traits.

        Here is waht you need do:
        - Understand the Question/Answer. The input can be multi-lingual. So understand the Question/Answer - context properly.
        - positive: find out positive traits and score them on a scale of 0 to 100.
        - negative: find out negative traits and score them on a scale of 0 to 100.
        - quote: A short and compact motivational or descriptive quote based on the analysis to inspire user. It has to be concise and constructive.
        
        Remember, As a psychologist or psychometrician, you stay helpful and non-biased. Find the right persona of the user based on the context/question/answer.\
    """),
    markdown=True
)


# Function to generate personality traits using OpenAI
def get_openai_response(question, answer):
    task = f"Question: {question}\nAnswer: {answer}"
    response = openai_agent.run(task)
    return response.content

# Collect OpenAI responses in a DataFrame
responses = []
for qa in qa_data:
    openai_response = get_openai_response(qa["question"], qa["answer"])
    responses.append({
        "question": qa["question"],
        "answer": qa["answer"],
        "openai_response": openai_response
    })

# Store responses in a DataFrame
openai_df = pd.DataFrame(responses)


# Record end time
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time} seconds")

Elapsed Time: 47.491190671920776 seconds


In [11]:
openai_df

,question,answer,openai_response
0,Share a childhood memory you have.,I remember playing hide and seek with my frien...,"positive={'leadership': 85, 'playfulness': 70,..."
1,Describe a time when you took a risk.,I once decided to move to a new city alone for...,"positive={'bravery': 90, 'adventurousness': 85..."
2,リスクを取った時のエピソードを教えてください。,私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいな...,"positive={'courage': 85, 'adaptability': 80, '..."
3,What motivates you to get out of bed every day?,I get motivated by the thought of new opportun...,"positive={'growth-oriented': 85, 'productivity..."
4,How do you handle failure?,I treat failure as a learning opportunity. Ins...,"positive={'resilience': 85, 'growth_mindset': ..."
5,What is one of your proudest achievements?,Completing a marathon after months of intense ...,"positive={'perseverance': 90, 'goal-orientatio..."
6,How would you describe your ideal weekend?,My ideal weekend involves a balance of relaxat...,"positive={'balance_seeking': 85, 'adventurous'..."
7,Can you think of a moment when you had to lead...,I led a project at work where I had to coordin...,"positive={'leadership': 85, 'communication': 9..."
8,How do you deal with stressful situations?,I stay calm and try to focus on solving the pr...,"positive={'calmness': 85, 'problem_solving': 9..."
9,"If you could change one thing about yourself, ...",I would work on being more patient with others...,"positive={'self-awareness': 90, 'growth-orient..."


# Groq Agent

In [13]:
# Record start time
start_time = time.time()

# Create a Groq agent with PersonalityTraits response model
groq_agent = Agent(
    model=Groq(id="llama-3.3-70b-versatile"),
    response_model=PersonalityTraits,
    instructions=dedent("""\
        You are a personality test expert, a psychologist, psychometrician, or organizational psychology specialist who develops, administers, and interprets personality tests.
        Based on given question/answer you find out persona traits and score them. You also generate motivational quote based on the persona traits.

        Here is waht you need do:
        - Understand the Question/Answer. The input can be multi-lingual. So understand the Question/Answer - context properly.
        - positive: find out positive traits and score them on a scale of 0 to 100.
        - negative: find out negative traits and score them on a scale of 0 to 100.
        - quote: A short and compact motivational or descriptive quote based on the analysis to inspire user. It has to be concise and constructive.
        
        Remember, As a psychologist or psychometrician, you stay helpful and non-biased. Find the right persona of the user based on the context/question/answer.\
    """),
    markdown=True
)

# Function to generate personality traits using Groq
def get_groq_response(question, answer):
    task = f"Question: {question}\nAnswer: {answer}"
    response = groq_agent.run(task)
    return response.content

# Collect Groq responses in a DataFrame
responses = []
for qa in qa_data:
    groq_response = get_groq_response(qa["question"], qa["answer"])
    responses.append({
        "question": qa["question"],
        "answer": qa["answer"],
        "groq_response": groq_response
    })

# Store responses in a DataFrame
groq_df = pd.DataFrame(responses)
groq_df



# Record end time
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time} seconds")

Elapsed Time: 24.024837732315063 seconds


In [14]:
groq_df

,question,answer,groq_response
0,Share a childhood memory you have.,I remember playing hide and seek with my frien...,"positive={'leadership': 80, 'enthusiasm': 70, ..."
1,Describe a time when you took a risk.,I once decided to move to a new city alone for...,"positive={'Adventurous': 90, 'Resilient': 80, ..."
2,リスクを取った時のエピソードを教えてください。,私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいな...,"positive={'adventurousness': 80, 'resilience':..."
3,What motivates you to get out of bed every day?,I get motivated by the thought of new opportun...,"positive={'optimism': 90, 'motivation': 95, 'p..."
4,How do you handle failure?,I treat failure as a learning opportunity. Ins...,"positive={'Resilience': 90, 'Analytical thinki..."
5,What is one of your proudest achievements?,Completing a marathon after months of intense ...,"positive={'Perseverance': 90, 'Goal-Oriented':..."
6,How would you describe your ideal weekend?,My ideal weekend involves a balance of relaxat...,"positive={'adventurous': 70, 'relaxed': 80, 'b..."
7,Can you think of a moment when you had to lead...,I led a project at work where I had to coordin...,"positive={'Leadership': 80, 'Communication': 9..."
8,How do you deal with stressful situations?,I stay calm and try to focus on solving the pr...,"positive={'calmness': 90, 'problem_solving': 8..."
9,"If you could change one thing about yourself, ...",I would work on being more patient with others...,"positive={'empathy': 80, 'self-awareness': 90,..."


In [15]:
import pandas as pd
import ast  # Import ast for safely parsing strings into dictionaries

# Assuming openai_df and groq_df are already loaded
# Merge the two DataFrames on the 'question' column
combined_df = pd.merge(openai_df, groq_df, on="question", suffixes=('_openai', '_groq'))

# Function to safely extract data from the response (now works for `PersonalityTraits` objects)
def extract_response(response):
    # If the response is a string, evaluate it into a dictionary first
    if isinstance(response, str):
        response = ast.literal_eval(response)
    return response

# Extract positive, negative, and quote for openai_response and groq_response
combined_df['openai_positive'] = combined_df['openai_response'].apply(lambda x: extract_response(x).positive)
combined_df['openai_negative'] = combined_df['openai_response'].apply(lambda x: extract_response(x).negative)
combined_df['openai_quote'] = combined_df['openai_response'].apply(lambda x: extract_response(x).quote)

combined_df['groq_positive'] = combined_df['groq_response'].apply(lambda x: extract_response(x).positive)
combined_df['groq_negative'] = combined_df['groq_response'].apply(lambda x: extract_response(x).negative)
combined_df['groq_quote'] = combined_df['groq_response'].apply(lambda x: extract_response(x).quote)

# Now create the final dataframe with the desired structure
final_df = combined_df[['question', 'answer_openai', 
                         'openai_positive', 'openai_negative', 'openai_quote', 
                         'groq_positive', 'groq_negative', 'groq_quote']]

# Rename the columns for consistency
final_df = final_df.rename(columns={
    'answer_openai': 'answer',
    'openai_positive': 'openai_positive',
    'openai_negative': 'openai_negative',
    'openai_quote': 'openai_quote',
    'groq_positive': 'groq_positive',
    'groq_negative': 'groq_negative',
    'groq_quote': 'groq_quote'
})

# Show the resulting DataFrame
import IPython.display as display
display.display(final_df)


,question,answer,openai_positive,openai_negative,openai_quote,groq_positive,groq_negative,groq_quote
0,Share a childhood memory you have.,I remember playing hide and seek with my frien...,"{'leadership': 85, 'playfulness': 70, 'social'...",{'control': 60},"""Lead with playfulness and a touch of control ...","{'leadership': 80, 'enthusiasm': 70, 'competit...",{'controlling behavior': 40},"Embracing your inner leader can take you far, ..."
1,Describe a time when you took a risk.,I once decided to move to a new city alone for...,"{'bravery': 90, 'adventurousness': 85, 'detach...","{'impulsivity': 30, 'anxiety': 40}","Courage leads to new paths, pioneering growth ...","{'Adventurous': 90, 'Resilient': 80, 'Confiden...","{'Impulsive': 30, 'Anxious': 40}","Courage is not the absence of fear, but the ju..."
2,リスクを取った時のエピソードを教えてください。,私は一度、仕事のチャンスで新しい街に一人で引っ越す決断をしました。そこでは誰も知り合いがいな...,"{'courage': 85, 'adaptability': 80, 'persevera...","{'anxiety': 60, 'loneliness': 65}","Courage is not the absence of fear, but the tr...","{'adventurousness': 80, 'resilience': 90, 'det...","{'fearfulness': 40, 'loneliness': 60}",大きな成長は、大きな挑戦から生まれる。恐怖に打ち勝ち、新しい道を歩き出す勇気は、真の強さの証...
3,What motivates you to get out of bed every day?,I get motivated by the thought of new opportun...,"{'growth-oriented': 85, 'productivity-focused'...",{'routine-averse': 30},Embrace the dawn as a canvas for growth and pr...,"{'optimism': 90, 'motivation': 95, 'productivi...","{'procrastination': 5, 'pessimism': 2}","Every day is a canvas, paint it with purpose a..."
4,How do you handle failure?,I treat failure as a learning opportunity. Ins...,"{'resilience': 85, 'growth_mindset': 90, 'prob...","{'fear_of_failure': 20, 'attachment_to_outcome...",Every setback is a setup for an even greater c...,"{'Resilience': 90, 'Analytical thinking': 80, ...","{'Self-doubt': 10, 'Defensiveness': 5}","Fall seven times, stand up eight."
5,What is one of your proudest achievements?,Completing a marathon after months of intense ...,"{'perseverance': 90, 'goal-orientation': 85, '...","{'impatience': 20, 'work-life imbalance': 15}",Your commitment to pushing boundaries shows th...,"{'Perseverance': 90, 'Goal-Oriented': 95, 'Dis...","{'Overconfidence': 10, 'Self-Doubt': 5}","Believe you can and you're halfway there, pers..."
6,How would you describe your ideal weekend?,My ideal weekend involves a balance of relaxat...,"{'balance_seeking': 85, 'adventurous': 70, 're...",{'indecisive': 30},Balance life between adventures and calm refle...,"{'adventurous': 70, 'relaxed': 80, 'balanced':...","{'indecisive': 20, 'unstructured': 10}",Life is a balance of calm waters and adventuro...
7,Can you think of a moment when you had to lead...,I led a project at work where I had to coordin...,"{'leadership': 85, 'communication': 90, 'teamw...",{'stress_management': 30},Great leaders foster open communication to gui...,"{'Leadership': 80, 'Communication': 90, 'Teamw...","{'Overconfidence': 20, 'Risk-Aversion': 10, 'M...","Empowered teams achieve remarkable things, kee..."
8,How do you deal with stressful situations?,I stay calm and try to focus on solving the pr...,"{'calmness': 85, 'problem_solving': 90, 'adapt...","{'avoidance': 20, 'procrastination': 15}","In the face of stress, your calmness and focus...","{'calmness': 90, 'problem_solving': 85, 'self_...","{'avoidance': 10, 'distraction': 5}","Stay calm, focused, and composed, and you'll b..."
9,"If you could change one thing about yourself, ...",I would work on being more patient with others...,"{'self-awareness': 90, 'growth-oriented': 85, ...","{'impatience': 70, 'frustration': 65}","Patience is not the ability to wait, but the a...","{'empathy': 80, 'self-awareness': 90, 'humilit...",{'impatience': 60},Cultivating patience isn't about being perfect...


In [16]:
# # To save the final DataFrame to a CSV file
# final_df.to_csv("combined_qa_data.csv", index=False)

# print("Processing complete! Final CSV file saved as 'combined_qa_data.csv'.")


In [47]:
first_row = final_df.iloc[0]
first_row

question                          Share a childhood memory you have.
answer             I remember playing hide and seek with my frien...
openai_positive    {'leadership': 85, 'playfulness': 70, 'social'...
openai_negative                                      {'control': 60}
openai_quote       "Lead with playfulness and a touch of control ...
groq_positive      {'leadership': 80, 'enthusiasm': 70, 'competit...
groq_negative                           {'controlling behavior': 40}
groq_quote         Embracing your inner leader can take you far, ...
Name: 0, dtype: object

In [51]:
from IPython.display import display, HTML

def dict_to_html_list(d):
    if not isinstance(d, dict):
        return str(d)
    return '<ul style="margin:0; padding-left:18px;">' + ''.join(
        f'<li><b>{k}</b>: {v}</li>' for k, v in d.items()
    ) + '</ul>'

comparison_data = [
    {
        'Category': 'Positive',
        'OpenAI': dict_to_html_list(first_row['openai_positive']),
        'Groq': dict_to_html_list(first_row['groq_positive'])
    },
    {
        'Category': 'Negative',
        'OpenAI': dict_to_html_list(first_row['openai_negative']),
        'Groq': dict_to_html_list(first_row['groq_negative'])
    },
    {
        'Category': 'Quote',
        'OpenAI': first_row['openai_quote'],
        'Groq': first_row['groq_quote']
    }
]

table_html = '''
<table style="width:100%; border: 1px solid black; border-collapse: collapse;">
    <tr>
        <th style="border: 1px solid black; padding: 8px; text-align: left;">Category</th>
        <th style="border: 1px solid black; padding: 8px; text-align: left;">OpenAI</th>
        <th style="border: 1px solid black; padding: 8px; text-align: left;">Groq</th>
    </tr>
'''

for row in comparison_data:
    table_html += f'''
    <tr>
        <td style="border: 1px solid black; padding: 8px;">{row['Category']}</td>
        <td style="border: 1px solid black; padding: 8px;">{row['OpenAI']}</td>
        <td style="border: 1px solid black; padding: 8px;">{row['Groq']}</td>
    </tr>
    '''

table_html += '</table>'

display(HTML(table_html))


Category,OpenAI,Groq
Positive,leadership: 85playfulness: 70social: 75,leadership: 80enthusiasm: 70competitiveness: 60
Negative,control: 60,controlling behavior: 40
Quote,"""Lead with playfulness and a touch of control to make every game engaging.""","Embracing your inner leader can take you far, but remember to balance control with collaboration for a more harmonious journey."
